In [13]:
import time
from collections import namedtuple

In [12]:
# Programming assignment 3.1.1&2 minimizing the weighted sum of completion times.

def score1(w, l):
    return w - l

def score2(w, l):
    return w/float(l)

def sortJobs (jobList, scoreFunc):
    scoreJobList = [(w,l, scoreFunc (w,l)) for w, l in jobList ]
    scoreJobList.sort(key=lambda tup: tup[0], reverse=True)   # sort by weight first to resolve ties
    scoreJobList.sort(key=lambda tup: tup[2], reverse=True)   # sort by score function
    return scoreJobList;

def loadData(fName):
    with open(fName,'r') as fileObj:
        lines = fileObj.readlines()
        numJobs = int (lines[0].strip())
        jobList = [ (int(line.split()[0]), int(line.split()[1])) for line in lines[1:]]
    return jobList

def sumCompletionTimes (sortedJobList):
    weightedSum = 0
    lengthSum = 0
    
    for job in sortedJobList:
        lengthSum += job[1]
        weightedSum += job[0] * lengthSum
        
    return weightedSum, lengthSum
    
def main(fName):
    %cd /Users/AM/Documents/_Courses/Algorithms/Misc
    jobList = loadData(fName);
    scoreJobList = sortJobs(jobList, score1)    #change to score1 or score2
    weightedSum, lengthSum = sumCompletionTimes (scoreJobList)
    print(weightedSum, lengthSum)
    
if __name__ == '__main__':
    main('text_3_1_jobs.txt')
    
#Score1: 69119377652 510289
#Score2: 67311454237 510289

/Users/AM/Documents/_Courses/Algorithms/misc
69119377652 510289


In [8]:
# Programming assignment 3.1.3 Prim's Minimum Spanning tree.


def getGraph(fPath = "text_3_1_edges.txt"):
    edgeList = []
    %cd /Users/AM/Documents/_Courses/Algorithms/Misc
    with open(fPath) as f:
        lines = f.read().splitlines()
        numNodes, numEdges = [int(el) for el in lines[0].split() ]
        for line in lines[1:]:
            node1, node2, edgeCost = [int(el) for el in line.split() ]
            edgeList.append((node1, node2, edgeCost))
            
    return edgeList, numNodes, numEdges


def prim(edgeList, numNodes):
    
    X = set()   # explored nodes
    V = set(range(1, numNodes+1))   # unexplored nodes
    E = set()   # set edges of MST
    totalCost = 0   # total sum of edge cost of the MST
    
    start = 1
    X.add(start)
    V.remove(start)
    
    while V:
        # look at all edges crossing the X and V sets
        lowestCost = float('inf')
        foundNodeX = None
        foundNodeV = None
        new_edge_in_X = None
        
        for edge in edgeList:
            if edge[0] in X and edge[1] in V:
                if edge[2] < lowestCost:
                    foundNodeX = edge[0]
                    foundNodeV = edge[1]
                    lowestCost = edge[2]
                    new_edge_in_X = edge
                    
            elif edge[1] in X and edge[0] in V:
                if edge[2] < lowestCost:
                    foundNodeX = edge[1]
                    foundNodeV = edge[0]
                    lowestCost = edge[2]
                    new_edge_in_X = edge

        
        #print("FoundNodeX: {0}, FoundNodeV: {1}".format(foundNodeX, foundNodeV))
        edgeList.remove(new_edge_in_X)
        X.add(foundNodeV)
        V.remove(foundNodeV)
        E.add((foundNodeX, foundNodeV, lowestCost))
        totalCost += lowestCost
        
    return E, totalCost
        
if __name__ == '__main__':
    edgeList, numNodes, numEdges = getGraph()
    t0 = time.time()
    E, totalCost = prim(edgeList, numNodes)
    t1 = time.time()

    print(f"Total cost: {totalCost}, time spent {t1 - t0}")

/Users/AM/Documents/_Courses/Algorithms/misc
Total cost: -3612829, time spent 0.14810800552368164


In [32]:
# Programming assignment 3.2 clustering

class Clust:
    
    def __init__ (self, file_name):
        %cd /Users/AM/Documents/_Courses/Algorithms/Misc
        with open(file_name) as file:
            lines = file.read().splitlines()
        self.n_nodes = int(lines[0])
        self.nodes = {}
        for line in lines[1:]:
            node_1, node_2, cost = [int(a) for a in line.split()]
            if node_1 in self.nodes.keys():
                self.nodes[node_1].append((node_2, cost))
            else:
                self.nodes[node_1] = [(node_2, cost)]

if __name__ == "__main__":
    c = Clust('text_3_2_clust.txt')
            
            


/Users/AM/Documents/_Courses/Algorithms/misc


In [48]:

s = '123'
a, b, c = [int(i) for i in '124']
print([int(i) for i in '124'])
type([int(i) for i in '124'])

[1, 2, 4]


list

In [9]:
numEdges

2184